<a href="https://colab.research.google.com/github/Syipmong/33-js-concepts/blob/master/leanbuild.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import zipfile
import os

zip_path = "/content/construction_dataset.zip"
extract_path = "/content/construction_dataset"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
  zip_ref.extractall(extract_path)
print("Extract Path", os.listdir(extract_path))

Extract Path ['.ipynb_checkpoints', 'captions.csv', 'construction_dataset', 'images']


Here we will make all our necessary imports

In [2]:
import os
import pandas as pd
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, TrainingArguments, Trainer
import torch
from datasets import Dataset


Here we will Load our dataset

In [3]:
folder_path = "/content/construction_dataset/images/"
csv_path = "/content/construction_dataset/captions.csv"

df = pd.read_csv(csv_path)
df["image_path"] = df['filename'].apply(lambda x: os.path.join(folder_path, x))

converting to huggingface dataset

In [4]:
dataset = Dataset.from_pandas(df[['image_path', 'caption']])
dataset = dataset.train_test_split(test_size=0.2)

Lets load our model

In [5]:
MODEL = "Salesforce/blip-image-captioning-large"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = BlipProcessor.from_pretrained(MODEL)
model = BlipForConditionalGeneration.from_pretrained(MODEL).to(DEVICE)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
def preprocess(examples):
  images = [Image.open(path).convert("RGB") for path in examples["image_path"]]
  captions = [str(caption) if pd.notna(caption) else "" for caption in examples['caption']]
  inputs = processor(images=images, text=captions, padding="max_length", return_tensors="pt")
  inputs["labels"] = inputs["input_ids"] # Add this line to include input_ids as labels
  return inputs

dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [10]:
training_args = TrainingArguments(
    output_dir = "./blip-finetuned-construction",
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 3,
    eval_strategy = "epoch", # Corrected parameter name
    save_strategy = "epoch",
    logging_dir = "./logs",
    logging_steps = 10,
    push_to_hub = False,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    tokenizer = processor
)

/tmp/ipython-input-10-3260013704.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()